# Imports

In [17]:
import pandas as pd 
import numpy as np 
import tensorflow as tf

import seqeval.metrics

In [2]:
## create dummy data 

In [39]:
labels = {
    'B-First':0,
    'I-First':1,
    'B-Second':2,
    'I-Second':3,
    'O':4,
    'PAD':-100
}

reversed_labels={v:k for k,v in labels.items()}

y_true = np.array([[0,1,1,1,4,2,3,3,3,-100,-100,-100],[-100,0,1,4,4,2,3,3,3,0,1,-100]])
y_pred = np.array([[0,1,1,1,4,2,3,3,4,4,4,4],[-100,0,1,1,2,2,3,3,0,0,0,4]])

# Accuracy

## seqeval experimentation

In [19]:
seqeval.metrics.accuracy_score(y_true,y_pred)

array([1. , 1. , 1. , 0.5, 0. , 1. , 1. , 1. , 0. , 0.5, 0. , 0. ])

In [54]:
def labeling_without_pad(y_true,y_pred,padding=-100):
    
    labels = {
    'B-First':0,
    'I-First':1,
    'B-Second':2,
    'I-Second':3,
    'O':4,
    'PAD':padding
    }

    reversed_labels={v:k for k,v in labels.items()}

    try:
        if not isinstance(y_true,numpy.array):
            y_true = y_true.numpy()
        if not isinstance(y_pred,numpy.array):
            y_pred = y_pred.numpy()
    except:
        "must be passed TF or NP"
    
    
    batch_size,seq_len = y_pred.shape
    labels, preds = [],[]
    
    for i in range(batch_size):
        labels_ , preds_ = [], []
        for j in range(seq_len):
            if y_true[i][j] !=-100:
                labels_.append(reversed_labels[y_true[i][j]])
                preds_.append(reversed_labels[y_pred[i][j]])
        labels.append(labels_)
        preds.append(preds_)
        
    return preds,labels
        

In [55]:
preds,labels=labeling_without_pad(y_true,y_pred)
print(seqeval.metrics.classification_report(preds,labels))

              precision    recall  f1-score   support

       First       0.33      0.20      0.25         5
      Second       0.00      0.00      0.00         3

   micro avg       0.20      0.12      0.15         8
   macro avg       0.17      0.10      0.12         8
weighted avg       0.21      0.12      0.16         8



In [56]:
seqeval.metrics.f1_score(labels,preds)

0.15384615384615385

In [75]:
seqeval.metrics.accuracy_score(labels,preds)

0.7368421052631579

## Accuracy masked subclassing

In [204]:
class AccuracyMasked(tf.keras.metrics.Metric):
    
    def __init__(self,class_to_ignore,name='accuracy_masked',**kwargs):
        super().__init__(name=name,**kwargs),
        self.class_to_ignore = class_to_ignore
        self.accs = self.add_weight(name = 'accs',initializer = 'zeros',dtype='float64')        
    
    def update_state(self,y_true,y_pred):
        #y_pred = tf.argmax(y_pred,axis=-1)
        #y_true = tf.argmax(y_true,axis=-1)#for y_pred and y_true to match
        #mask = tf.cast(tf.not_equal(y_true,self.class_to_ignore),'int32') #create a mask
        #matches = tf.cast(tf.equal(y_true,y_pred),'int32')*mask #calculate the matches ignoring the masking
        #accuracy = tf.math.reduce_sum(matches,axis=-1)/tf.maximum(tf.math.reduce_sum(mask,axis=-1),1)
        accuracy=tf.math.reduce_sum(tf.cast(y_true == y_pred,'int32')*tf.cast(y_true != -100,'int32'))/tf.math.reduce_sum(tf.cast(y_true != -100,'int32'))
        self.accs.assign_add(tf.math.reduce_mean(accuracy))
        
    
    def result(self):
        return self.accs #tf.math.reduce_mean(accuracy)


In [205]:
accuracymasked = AccuracyMasked(-100)

accuracymasked(y_pred,y_true)

<tf.Tensor: shape=(), dtype=float64, numpy=0.6086956521739131>

In [181]:
tf.math.reduce_sum(tf.cast(y_true == y_pred,'int32')*tf.cast(y_true != -100,'int32'))/tf.math.reduce_sum(tf.cast(y_true != -100,'int32'))

<tf.Tensor: shape=(), dtype=float64, numpy=0.7368421052631579>

In [99]:
mask = tf.cast(tf.not_equal(y_true,-100),'int32') #create a mask
matches = tf.cast(tf.equal(y_true,y_pred),'int32')*mask #calculate the matches ignoring the masking
accuracy = tf.math.reduce_sum(matches,axis=-1)/tf.maximum(tf.math.reduce_sum(mask,axis=-1),1)

In [103]:
tf.math.reduce_mean(accuracy)

<tf.Tensor: shape=(), dtype=float64, numpy=0.7444444444444445>

## Accuracy homemade via function

In [208]:
##SIMPLER ACCURACY definition

def accuracy_masked_func(y_true,y_pred):
    #y_pred = tf.argmax(y_pred,axis=-1)
    #y_true = tf.argmax(y_true,axis=-1)#for y_pred and y_true to match
    mask = tf.cast(y_true != -100,'int32') #create a mask
    matches = tf.cast(tf.equal(y_true,y_pred),'int32')*mask #calculate the matches ignoring the masking
    accuracy = tf.math.reduce_sum(matches,axis=-1)/tf.maximum(tf.math.reduce_sum(mask,axis=-1),1)
    
    return accuracy

In [207]:
accuracy_masked_func(y_true,y_pred)

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.88888889, 0.6       ])>

In [210]:
accuracy_masked = tf.keras.metrics.MeanMetricWrapper(fn=accuracy_masked_func)

In [211]:
accuracy_masked(y_true,y_pred)

<tf.Tensor: shape=(), dtype=float32, numpy=0.7444445>

# Evaluation metric

In [ ]:
## Pseudo code

# 1- calculate the overlaps
# 2- for each pred : determine if it is a TP/FN/TN regarding the opvermap 
# 3- compute f1 score for each class 
# 4- compute f1 macro score across all classes 

In [213]:
df = pd.read_csv('../raw_data/preprocessed.csv',nrows=100)

In [214]:
df.head()

,id,predictionstring,label,essays
0,0000D23A521A,"['0', '1', '2', '3', '4', '5', '6', '7', '8', ...","['B-Position', 'I-Position', 'I-Position', 'I-...","Some people belive that the so called ""face"" o..."
1,00066EA9880D,"['0', '1', '2', '3', '4', '5', '6', '7', '8', ...","['B-Lead', 'I-Lead', 'I-Lead', 'I-Lead', 'I-Le...",Driverless cars are exaclty what you would exp...
2,000E6DE9E817,"['2', '3', '4', '5', '6', '7', '8', '10', '11'...","['B-Position', 'I-Position', 'I-Position', 'I-...",Dear: Principal\n\nI am arguing against the po...
3,001552828BD0,"['0', '1', '2', '3', '4', '5', '6', '7', '8', ...","['B-Lead', 'I-Lead', 'I-Lead', 'I-Lead', 'I-Le...",Would you be able to give your car up? Having ...
4,0016926B079C,"['0', '1', '2', '3', '4', '5', '6', '7', '8', ...","['B-Position', 'I-Position', 'I-Position', 'I-...",I think that students would benefit from learn...


In [215]:
df_train = pd.read_csv('../raw_data/train.csv',nrows=100)

In [216]:
df_train.head()

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...
